In [502]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

from asset_data_module import read_close_prices_all_merged
from features import make_feature_windows
tf.keras.utils.set_random_seed(42)


In [ ]:
markets = ['dow30', 'commodities']
# markets = ['dow30']
# markets = ['commodities']
start_date, end_date = "2022-01-01", "2025-11-28"
# start_date, end_date = "2024-06-01", "2025-11-28"

_, close_df = read_close_prices_all_merged(markets, after_date=start_date)
close_df = close_df.loc[:end_date]

rolling = make_feature_windows(
    close_prices=close_df,
    lookback=60,
    horizon=1,
    days_per_week=2
)
close_df.shape, len(rolling)

((981, 30), 430)

In [504]:
print(rolling[0]['past_weekly_returns'].shape)
rolling[0]['past_weekly_returns'].head()

(60, 30)


,dow30:AAPL,dow30:AMGN,dow30:AXP,dow30:BA,dow30:CAT,dow30:CRM,dow30:CSCO,dow30:CVX,dow30:DIS,dow30:DOW,...,dow30:MSFT,dow30:NKE,dow30:PFE,dow30:PG,dow30:RTX,dow30:TRV,dow30:UNH,dow30:V,dow30:VZ,dow30:WMT
0,-0.039733,-0.006861,0.020885,0.024756,0.059772,-0.115169,-0.040796,0.024517,-0.010066,0.025691,...,-0.056440,-0.014805,-0.018169,0.008010,0.029457,0.025483,-0.025385,-0.006479,0.029685,-0.005060
1,-0.015846,0.009504,0.017646,0.011340,0.020003,0.002807,0.014002,0.022731,0.016868,0.012772,...,-0.007424,-0.033084,0.001617,-0.008992,0.009666,0.039697,-0.065594,-0.013915,0.016041,0.006717
2,0.016761,0.022147,0.005718,0.002410,-0.019094,0.028200,0.020082,0.023242,0.000380,0.001353,...,0.002989,-0.043421,0.017259,-0.025390,0.003091,-0.018701,0.022424,-0.011963,-0.012056,-0.004773
3,-0.016645,-0.006606,-0.011988,0.035828,0.031550,-0.026799,-0.014047,-0.009185,-0.015639,0.012760,...,-0.032853,-0.004735,-0.020494,-0.002335,0.009217,-0.000674,-0.003353,0.002888,-0.001307,0.008769
4,-0.013977,0.012826,-0.041955,0.004945,0.012564,-0.011305,-0.029203,0.020143,-0.020605,0.004328,...,-0.007079,-0.017806,-0.026084,-0.009904,-0.008886,0.003184,-0.013873,0.003297,-0.001683,-0.020487


In [505]:
rolling[0]['X_feat'].head()

,mom_1w,mom_4w,mom_12w,vol_1w,vol_4w,sharpe_1w,sharpe_4w,vol_ratio,max_drawdown
dow30:AAPL,0.549830,0.655922,0.334903,-1.335888,0.073495,3.354612,0.664756,-1.441571,-0.377131
dow30:AMGN,-0.395536,0.117647,-0.007335,-1.336021,-1.307819,1.230191,0.538996,-1.304322,1.442540
dow30:AXP,-0.766660,-0.432284,-0.666186,0.774774,1.454327,-0.617898,-0.598703,0.164282,-0.591565
dow30:BA,-0.009124,2.586184,1.253099,2.017293,2.145951,-0.577235,1.511861,0.233711,-2.413837
dow30:CAT,-1.736866,-2.056154,-1.342731,2.418059,1.560400,-0.715582,-1.877875,1.526759,0.094049


In [506]:
rolling[0]['y_ret'].head(3)

dow30:AAPL   -0.030242
dow30:AMGN   -0.005488
dow30:AXP    -0.014661
dtype: float64

In [507]:
def panel_from_windows(windows, x_key="past_weekly_returns", y_key="y_ret"):
    X_list, y_list = [], []
    meta_rows = []

    for w_idx, w in enumerate(windows):
        if x_key == 'past_weekly_returns':
            X_df = w[x_key].T          # assets x n_lookback (weeks) (DataFrame)
        elif x_key == 'X_feat':
            X_df = w[x_key]          # assets x n_features (DataFrame)
        y_ser = w[y_key]           # assets (Series or array)

        if not isinstance(y_ser, pd.Series):
            y_ser = pd.Series(y_ser, index=X_df.index)

        assets = X_df.index.intersection(y_ser.index)
        Xw = X_df.loc[assets].to_numpy(dtype=np.float32)
        yw = y_ser.loc[assets].to_numpy(dtype=np.float32)

        mask = np.isfinite(Xw).all(axis=1) & np.isfinite(yw)
        Xw, yw = Xw[mask], yw[mask]
        assets_kept = assets.to_numpy()[mask]

        X_list.append(Xw)
        y_list.append(yw)

        t0, t1 = w.get("t0", None), w.get("t1", None)
        for a in assets_kept:
            meta_rows.append((w_idx, a, t0, t1))

    X = np.vstack(X_list) ## weeks*assets x n_lookback/n_features
    y = np.concatenate(y_list)
    meta = pd.DataFrame(meta_rows, columns=["window_idx", "asset", "t0", "t1"])
    return X, y, meta

In [508]:
X_past_returns, y, meta = panel_from_windows(rolling, x_key='past_weekly_returns')
X, y, meta = panel_from_windows(rolling, x_key='past_weekly_returns')
# X, y, meta = panel_from_windows(rolling, x_key='X_feat')
X.shape, y.shape, meta.head() ## len(rolling)*n_asset -- each row is a feature set -- to predict y

((12900, 60),
 (12900,),
    window_idx       asset         t0         t1
 0           0  dow30:AAPL 2022-06-24 2022-06-28
 1           0  dow30:AMGN 2022-06-24 2022-06-28
 2           0   dow30:AXP 2022-06-24 2022-06-28
 3           0    dow30:BA 2022-06-24 2022-06-28
 4           0   dow30:CAT 2022-06-24 2022-06-28)

In [509]:
windows = rolling  # your rolling list

W = meta["window_idx"].nunique()
split_w = int(0.8 * W)

train_mask = (meta["window_idx"] < split_w).values
test_mask  = (meta["window_idx"] >= split_w).values

X_train_raw, y_train = X[train_mask], y[train_mask]
X_test_raw,  y_test  = X[test_mask],  y[test_mask]
X_past_returns_test_raw = X_past_returns[test_mask]

# small validation from the tail of the training windows
val_w = max(int(0.1 * split_w), 1)
val_start = split_w - val_w
val_mask = ((meta["window_idx"] >= val_start) & (meta["window_idx"] < split_w)).values
tr2_mask = (meta["window_idx"] < val_start).values

X_tr_raw, y_tr = X[tr2_mask], y[tr2_mask]
X_va_raw, y_va = X[val_mask], y[val_mask]

scaler = StandardScaler()
X_tr = scaler.fit_transform(X_tr_raw).astype(np.float32)
X_va = scaler.transform(X_va_raw).astype(np.float32)
X_te = scaler.transform(X_test_raw).astype(np.float32)

y_tr = y_tr.astype(np.float32)
y_va = y_va.astype(np.float32)
y_te = y_test.astype(np.float32)

## SCALE Y
y_mean = y_tr.mean()
y_std  = y_tr.std() + 1e-8

y_tr_s = ((y_tr - y_mean) / y_std).astype(np.float32)
y_va_s = ((y_va - y_mean) / y_std).astype(np.float32)
float(y_std)

# X_tr = np.tanh(X_tr)
# X_va = np.tanh(X_va)
# X_te = np.tanh(X_te)

0.015860024839639664

In [510]:
X_tr.shape, X_va.shape, X_te.shape, y_tr.shape, y_va.shape, y_te.shape, f"{y_te.shape[0]/close_df.shape[1]} test periods"

((9300, 60),
 (1020, 60),
 (2580, 60),
 (9300,),
 (1020,),
 (2580,),
 '86.0 test periods')

In [511]:
def build_mlp(in_dim, hidden=(64, 32), dropout=0.1, lr=1e-3):
    inputs = keras.Input(shape=(in_dim,))
    x = inputs
    for h in hidden:
        x = layers.Dense(h, activation="relu",)(x) #kernel_regularizer=regularizers.l2(1e-4)
        x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mse",
        metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
    )
    return model

model = build_mlp(in_dim=X_tr.shape[1], hidden=(32, 16), dropout=0.1, lr=1e-4)
# model = build_mlp(in_dim=X_tr.shape[1], hidden=(15, 15, 15, 15, 15, 15), dropout=0.0, lr=1e-4)
# model = build_mlp(in_dim=X_tr.shape[1], hidden=(15, 15, 15, 15, 15, 15), dropout=0.0, lr=0.01)

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=100, restore_best_weights=True
    )
]

history = model.fit(
    # X_tr, y_tr,
    X_tr, y_tr_s,   ## Scaled y
    # validation_data=(X_va, y_va),
    validation_data=(X_va, y_va_s),   ## Scaled y
    # epochs=150,
    epochs=100,
    # batch_size=256,
    batch_size=100,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.4256 - mae: 0.8625 - val_loss: 1.4019 - val_mae: 0.8437
Epoch 2/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step - loss: 1.2952 - mae: 0.8165 - val_loss: 1.3500 - val_mae: 0.8231
Epoch 3/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - loss: 1.2679 - mae: 0.8081 - val_loss: 1.3139 - val_mae: 0.8075
Epoch 4/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - loss: 1.2196 - mae: 0.7897 - val_loss: 1.2868 - val_mae: 0.7958
Epoch 5/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - loss: 1.1870 - mae: 0.7735 - val_loss: 1.2660 - val_mae: 0.7860
Epoch 6/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step - loss: 1.1455 - mae: 0.7593 - val_loss: 1.2522 - val_mae: 0.7792
Epoch 7/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - loss: 1.1266 - mae: 0.7503 - val_loss: 1.2405 - val_mae: 0.7732
Epoch 8/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - loss: 1.1093 - mae: 0.7431 - val_loss: 1.2315 - val_mae: 0.7688
Epoch 9/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/st

In [512]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

def prediction_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    sign_acc = (np.sign(y_true) == np.sign(y_pred)).mean()
    corr = np.corrcoef(y_true, y_pred)[0, 1] if len(y_true) > 1 else np.nan
    
    return {"MSE": round(mse, 5), "MAE": round(mae, 5), "R2": round(r2, 5), "SignAcc": round(float(sign_acc), 5), "Corr": round(float(corr), 5)}


In [513]:
y_pred_sm = X_past_returns_test_raw.astype(np.float32).mean(axis=1)
# X_te_raw = scaler.inverse_transform(X_te)
# y_pred_sm = X_te_raw.mean(axis=1)
print("Sample Mean")
print(prediction_metrics(y_te, y_pred_sm))

# y_pred = model.predict(X_te, batch_size=1024).squeeze()
## Scaled y:
pred_s = model.predict(X_te, batch_size=1024).squeeze()
y_pred = y_mean + y_std * pred_s

print("MLP")
print(prediction_metrics(y_te, y_pred))

print(y_te.mean())

Sample Mean
{'MSE': 0.00049, 'MAE': 0.01368, 'R2': -0.05323, 'SignAcc': 0.48295, 'Corr': -0.08989}
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
MLP
{'MSE': 0.00049, 'MAE': 0.01385, 'R2': -0.04723, 'SignAcc': 0.49535, 'Corr': 0.00419}
0.0022442322


In [514]:
def baseline_sample_mean(test_window):
    # past_weekly_returns: (lookback periods) x (assets)
    return test_window["past_weekly_returns"].mean(axis=0)  # pd.Series indexed by asset

Walk-Forward Evaluation

In [515]:
def walk_forward_eval_mlp(
    rolling,
    train_len=150,          # number of windows to train on each step
    x_key="X_feat",
    y_key="y_ret",
    hidden=(64, 32),
    dropout=0.1,
    lr=3e-4,
    epochs=150,
    batch_size=256,
    seed=42
):
    tf.keras.utils.set_random_seed(seed)

    all_rows = []
    week_metrics = []

    for i in range(train_len, len(rolling)):
        train_windows = rolling[i-train_len:i]
        test_window   = rolling[i]

        # --- build train panel ---
        X_train, y_train, meta = panel_from_windows(train_windows, x_key=x_key, y_key=y_key)
        if X_train.shape[0] == 0:
            continue

        # --- scaler on TRAIN only ---
        scaler = StandardScaler()
        X_train_sc = scaler.fit_transform(X_train).astype(np.float32)

        # --- build test cross-section ---
        X_test_df = test_window[x_key]            # assets x features
        y_test = test_window[y_key]
        if not isinstance(y_test, pd.Series):
            y_test = pd.Series(y_test, index=X_test_df.index)

        assets = X_test_df.index.intersection(y_test.index).sort_values()
        X_test = X_test_df.loc[assets].to_numpy(np.float32)
        y_true = y_test.loc[assets].to_numpy(np.float32)

        mask = np.isfinite(X_test).all(axis=1) & np.isfinite(y_true)
        assets = assets[mask]
        X_test = X_test[mask]
        y_true = y_true[mask]

        X_test_sc = scaler.transform(X_test).astype(np.float32)

        # --- train model (fresh each step) ---
        model = build_mlp(in_dim=X_train_sc.shape[1], hidden=hidden, dropout=dropout, lr=lr)
        es = keras.callbacks.EarlyStopping(monitor="loss", patience=15, restore_best_weights=True)

        model.fit(
            X_train_sc, y_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            callbacks=[es]
        )

        # --- predict ---
        y_pred = model.predict(X_test_sc, batch_size=1024, verbose=0).squeeze()

        # --- baseline: sample mean of past period returns ---
        y_pred_sm_ser = baseline_sample_mean(test_window).loc[assets]
        y_pred_sm = y_pred_sm_ser.to_numpy(np.float32)

        # store per-asset predictions
        for a, yt, yp, ypsm in zip(assets, y_true, y_pred, y_pred_sm):
            all_rows.append(
                {"window_idx": i, "asset": a, "y": float(yt), "pred_mlp": float(yp), "pred_sm": float(ypsm)}
            )

        # per-window metrics (cross-section)
        m_mlp = prediction_metrics(y_true, y_pred)
        m_sm  = prediction_metrics(y_true, y_pred_sm)
        week_metrics.append({"window_idx": i, **{f"mlp_{k}": v for k,v in m_mlp.items()},
                                **{f"sm_{k}": v for k,v in m_sm.items()}})

    preds_df = pd.DataFrame(all_rows)
    week_df  = pd.DataFrame(week_metrics)

    # pooled metrics over all (window, asset) test points
    pooled_mlp = prediction_metrics(preds_df["y"], preds_df["pred_mlp"])
    pooled_sm  = prediction_metrics(preds_df["y"], preds_df["pred_sm"])

    return preds_df, week_df, pooled_mlp, pooled_sm

In [516]:
# preds_df, week_df, pooled_mlp, pooled_sm = walk_forward_eval_mlp(
#     rolling=rolling,
#     train_len=100,   # e.g., last 52 periods (with days_per_week=2 that's ~104 trading days)
#     x_key="X_feat",
#     y_key="y_ret",
#     hidden=(16,8)
# )

# print("Pooled MLP:", pooled_mlp)
# print("Pooled  SM:", pooled_sm)
# print("Weekly win-rate (MAE):", (week_df["mlp_MAE"] < week_df["sm_MAE"]).mean())

In [517]:
def walk_forward_export_mlp(
    rolling,
    markets: list[str],
    model_name: str = "MLP",
    year: int = 2025,
    train_len: int = 200,
    x_key: str = "past_weekly_returns",
    y_key: str = "y_ret",
    hidden=(32, 16),
    dropout: float = 0.1,
    lr: float = 3e-4,
    epochs: int = 150,
    batch_size: int = 256,
    seed: int = 42,
    out_dir: str = "data/prediction",
    date_key_candidates=("t1", "t0", "date", "Date"),
):
    tf.keras.utils.set_random_seed(seed)
    os.makedirs(out_dir, exist_ok=True)
    markets_str = "-".join(markets)

    rows_pred = []
    rows_true = []

    def get_window_date(w):
        for k in date_key_candidates:
            if k in w:
                d = pd.to_datetime(w[k])
                return d
        return None  # if no date info

    # find first index in 2025 (so we "start walking forward on 2025")
    first_2025_idx = None
    for i in range(len(rolling)):
        d = get_window_date(rolling[i])
        if isinstance(d, pd.Timestamp) and d.year == year:
            first_2025_idx = i
            break

    if first_2025_idx is None:
        raise ValueError(f"No windows found with year={year}. Check your rolling windows date keys.")

    # walk-forward loop: start at first 2025 window, but still need train_len history
    start_i = max(train_len, first_2025_idx)
    print("START i ")
    print(first_2025_idx)
    print(start_i)
    print(len(rolling))
    for i in range(start_i, len(rolling)):
        train_windows = rolling[i-train_len:i]
        test_window   = rolling[i]

        period = get_window_date(test_window)
        if isinstance(period, pd.Timestamp):
            if period.year != year:
                continue
        else:
            # if no dates, we cannot filter by year; safest is to skip
            continue

        # --- build train panel ---
        X_train, y_train, _ = panel_from_windows(train_windows, x_key=x_key, y_key=y_key)
        if X_train.shape[0] == 0:
            continue

        # --- scaler on TRAIN only ---
        scaler = StandardScaler()
        X_train_sc = scaler.fit_transform(X_train).astype(np.float32)

        # --- build test cross-section ---
        X_test_df = test_window[x_key].T  # assets x features
        y_test = test_window[y_key]
        if not isinstance(y_test, pd.Series):
            y_test = pd.Series(y_test, index=X_test_df.index)

        assets = X_test_df.index.intersection(y_test.index).sort_values()
        if len(assets) == 0:
            continue

        X_test = X_test_df.loc[assets].to_numpy(np.float32)
        y_true = y_test.loc[assets].to_numpy(np.float32)

        mask = np.isfinite(X_test).all(axis=1) & np.isfinite(y_true)
        assets = assets[mask]
        X_test = X_test[mask]
        y_true = y_true[mask]

        if len(assets) == 0:
            continue

        X_test_sc = scaler.transform(X_test).astype(np.float32)

        # --- train model (fresh each step) ---
        keras.backend.clear_session()
        model = build_mlp(in_dim=X_train_sc.shape[1], hidden=hidden, dropout=dropout, lr=lr)

        # (optional) early stopping; monitor val_loss only if you pass validation_data
        es = keras.callbacks.EarlyStopping(monitor="loss", patience=15, restore_best_weights=True)

        model.fit(
            X_train_sc, y_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            callbacks=[es]
        )

        # --- predict ---
        y_pred = model.predict(X_test_sc, batch_size=1024, verbose=0).squeeze()

        # store long rows (period, asset)
        for a, yt, yp in zip(assets, y_true, y_pred):
            rows_true.append({"period": period, "asset": a, "true": float(yt)})
            rows_pred.append({"period": period, "asset": a, "pred": float(yp)})

    # --- long -> wide ---
    pred_long = pd.DataFrame(rows_pred)
    true_long = pd.DataFrame(rows_true)

    if pred_long.empty:
        raise ValueError("No predictions were produced. Check train_len, date keys, or rolling coverage.")

    expected_df = pred_long.pivot(index="period", columns="asset", values="pred").sort_index()
    true_df     = true_long.pivot(index="period", columns="asset", values="true").sort_index()

    # align columns (union) so shapes match
    all_cols = expected_df.columns.union(true_df.columns)
    expected_df = expected_df.reindex(columns=all_cols)
    true_df     = true_df.reindex(columns=all_cols)

    errors_df = true_df - expected_df  # error = true - pred

    # --- save ---
    p_exp = os.path.join(out_dir, f"{model_name}_{markets_str}_expected_returns.csv")
    p_true = os.path.join(out_dir, f"{model_name}_{markets_str}_true_returns.csv")
    p_err = os.path.join(out_dir, f"{model_name}_{markets_str}_errors.csv")

    expected_df.to_csv(p_exp)
    true_df.to_csv(p_true)
    errors_df.to_csv(p_err)

    print("Saved:")
    print(" ", p_exp, expected_df.shape)
    print(" ", p_true, true_df.shape)
    print(" ", p_err, errors_df.shape)

    return expected_df, true_df, errors_df

In [518]:
expected_2025, true_2025, errors_2025 = walk_forward_export_mlp(
    rolling=rolling,
    markets=markets,     
    model_name="MLP",
    year=2025,
    train_len=200,
    x_key="past_weekly_returns",
    y_key="y_ret",
    hidden=(32, 16),
    dropout=0.1,
    lr=1e-4,
    epochs=150,
    batch_size=256,
    out_dir="data/prediction",
)

START i 
316
316
430
Saved:
  data/prediction/MLP_dow30_expected_returns.csv (114, 30)
  data/prediction/MLP_dow30_true_returns.csv (114, 30)
  data/prediction/MLP_dow30_errors.csv (114, 30)
